Notebook to implement u-net background subtraction

In [9]:
%matplotlib inline

import os
import wandb
import random
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn 
import torchvision.models as models
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image
import numpy as np


In [10]:
# Choose device to run on

print(torch.__version__)

if(torch.backends.mps.is_available()):
    device = "mps"
elif(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu"

print(f"Running on {device}...")


2.5.1
Running on cuda...


In [11]:
%pip install wandb -qU

Note: you may need to restart the kernel to use updated packages.


In [12]:
# set seed value for repeatability

seed = 42
random.seed(seed)  # Python's random module
torch.manual_seed(seed)  # PyTorch CPU/GPU seed
np.random.seed(seed)  # NumPy random seed

In [13]:
# Start loading up the data

# TODO: update base_path to point to wherever your data is stored
base_path = "../../Data/data/"

train_features = pd.read_csv(f"{base_path}train_features.csv", index_col="id")
test_features = pd.read_csv(f"{base_path}test_features.csv", index_col="id")
train_labels = pd.read_csv(f"{base_path}train_labels.csv", index_col="id")



In [14]:
# Add a 'filepath' column with the full path to each image
# Subdirectories for train and test images
train_images_path = os.path.join(base_path, "train_features")
test_images_path = os.path.join(base_path, "test_features")

train_features['filepath'] = train_features.index.map(
    lambda img_id: os.path.join(train_images_path, f"{img_id}.jpg"))

test_features['filepath'] = test_features.index.map(
    lambda img_id: os.path.join(test_images_path, f"{img_id}.jpg"))

In [ ]:
species_labels = sorted(train_labels.columns.unique())


We start diverging from the "standard" example provided by the competition here. Let's pull out all the blank images from the train dataset. Then we'll test/train split on that for training the U-Net.

In [22]:
blank_training_labels = train_labels[train_labels["blank"]==1]

blank_training_images = train_features.loc[blank_training_labels.index]


Since we're only using one class here, this is more of an unsupervised activity than a supervised one. As such, we only have

In [ ]:
X_train, X_test = train_test_split(blank_training_images, test_size=0.2, random_state=42)

In [ ]:
class BackgroundDataset(Dataset):

    def __init__(self, train_features, transform = None, device=device):
        self.data = train_features, 
        self.device = device,
        self.transform = transform

    def __getitem__(self, index):
        image_id = self.data.index[index]
        image = Image.open(self.data.iloc[index]["filepath"]).convert("RGB")

        if self.transform:
            image = self.transform(image)

        sample = {"image_id": image_id, "image": image}

        return sample
    
    def __len__(self):
        return len(self.data)
    

background_transform = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor()
])



# Create datasets
train_dataset = BackgroundDataset(X_train, transform=background_transform, device=device)
test_dataset =  BackgroundDataset(X_test, transform= background_transform, device=device)

pin = False if device=="cuda" else True

train_loader = DataLoader(
    train_dataset, 
    batch_size = BATCH_SIZE, 
    shuffle = True,
    pin_memory = pin
)

In [ ]:
# Create datasets
train_dataset = BackgroundDataset(X_train, transform=background_transform, device=device)
test_dataset =  BackgroundDataset(X_test, transform= background_transform, device=device)


# Choose batch size here:
BATCH_SIZE = 32

pin = False if device=="cuda" else True
train_loader = DataLoader(
    train_dataset, 
    batch_size = BATCH_SIZE, 
    shuffle = True,
    pin_memory = pin
)

test_loader = DataLoader(
    test_dataset, 
    batch_size = BATCH_SIZE,
    shuffle = True,
    pin_memory = pin
)



Ok, lets start working on building the U-Net itself. 

In [ ]:
# guided by: https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_parts.py

class DownConvolve(nn.Module):

    def __init__(self, in_channels, out_channels):

        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.down = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.in_cannels, self.out_channels, kernel_size=3, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        def forward(self, x):
             return self.down(x)

class UpConvolve(nn.Module):

    def __init__(self, in_channels, out_channels):

        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.up = nn.Sequential(
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )


In [ ]:


class UnetModel(nn.Module):

    def __init__(self):
        super().__init__()

